In [1]:
!nvidia-smi

Thu Feb  1 02:29:28 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A4000    Off  | 00000000:00:05.0 Off |                  Off |
| 41%   35C    P8    14W / 140W |      1MiB / 16376MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
!pip install accelerate bitsandbytes langchain

In [2]:
%cd /content
!apt-get update -y 
!apt-get -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/resolve/main/model-00001-of-00002.safetensors -d /content/models/Llama-2-7b-chat-hf -o model-00001-of-00002.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/resolve/main/model-00002-of-00002.safetensors -d /content/models/Llama-2-7b-chat-hf -o model-00002-of-00002.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/raw/main/model.safetensors.index.json -d /content/models/Llama-2-7b-chat-hf -o model.safetensors.index.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/raw/main/special_tokens_map.json -d /content/models/Llama-2-7b-chat-hf -o special_tokens_map.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/resolve/main/tokenizer.model -d /content/models/Llama-2-7b-chat-hf -o tokenizer.model
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/raw/main/tokenizer_config.json -d /content/models/Llama-2-7b-chat-hf -o tokenizer_config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/raw/main/config.json -d /content/models/Llama-2-7b-chat-hf -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/Llama-2-7b-chat-hf/raw/main/generation_config.json -d /content/models/Llama-2-7b-chat-hf -o generation_config.json


/content
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1581 B]
Get:2 https://deb.nodesource.com/node_16.x focal InRelease [4583 B]            
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1345 kB]
Get:4 https://deb.nodesource.com/node_16.x focal/main amd64 Packages [776 B]   
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]        
Get:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 Packages [35.5 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]     
Get:11 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [29.7 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-se

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
import torch

from transformers import BitsAndBytesConfig
from torch import bfloat16

# Our 4-bit configuration to load the LLM with less GPU memory
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_quant_type='nf4',  # Normalized float 4
    bnb_4bit_use_double_quant=True,  # Second quantization after the first
    bnb_4bit_compute_dtype=bfloat16  # Computation type
)

model_id = "/content/models/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_id, revision='main', use_fast=True, trust_remote_code=True)
model_config = AutoConfig.from_pretrained(
    model_id
)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


In [10]:

from transformers import TextIteratorStreamer
from threading import Thread

def chat(messages, model, tokenizer, **kwargs):
    input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
    input_ids = input_ids.to("cuda")
    streamer = TextIteratorStreamer(tokenizer, timeout=60.0, skip_prompt=True, skip_special_tokens=True)
    
    gen_kwargs = {"input_ids": input_ids, "max_length": 512, "eos_token_id": tokenizer.eos_token_id, "streamer": streamer, 'do_sample': True, 'temperature': None, 'top_k': 10}
    thread = Thread(target=model.generate, kwargs=gen_kwargs)
    thread.start()
    return streamer

def display_chat(prompt, system=None, ret=False, **kwargs):
    messages = [{'role': 'user', 'content': prompt}]
    if system is not None:
        messages = [{'role': 'system', 'content': system}] + messages

    message = ""
    for word in chat(messages, model, tokenizer):
        print(word, end='')
        message += word
    
    if ret:
        return message


In [9]:
PROMPT_TEMPLATE = """"You are an math expert which solve questions best as you can.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
OBSERVATION: the result of the action
... (this Thought/Action/Action Input/OBSERVATION can be repeated zero or more times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question.

Question: {query}"""
display_chat(PROMPT_TEMPLATE.format(query="how much is 8 + 2 / 2 * 2?"))


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


 Sure, I'd be happy to help you with that question!

Thought: Oh, this is a fun one! Let's see...

Action: I'll start by multiplying 2 * 2, which gives me 4.
Action Input: 4

Thought: Great, now I have 4 as my intermediate result. How can I use this to solve the original question?

OBSERVATION: Hmm, if I have 4 and I want to find 8 + 2, I can simply add 4 and 2, which gives me 6.

Thought: Ah, I see! So the final answer is 6.

Final Answer: 6

Therefore, the answer to the question "how much is 8 + 2 / 2 * 2?" is 6.

In [11]:
iterations = 3
results = []
for _ in range(iterations):
    ret = display_chat(PROMPT_TEMPLATE.format(query="how much is 8 + 2 / 2 * 2?"), ret=True)
    results.append(ret)
results

 Sure, I'd be happy to help you solve this equation! Here's the format you requested:

Question: How much is 8 + 2 / 2 * 2?

Thought: Let's start by adding 8 and 2: 8 + 2 = 10.

Action: Divide 10 by 2: 10 / 2 = 5.

Action Input: 5.

OBSERVATION: 5 is the result of dividing 10 by 2.

Thought: Great, now we have 5 as a result. Let's use it to simplify the equation: 8 + 2 / 2 * 2 = 8 + 2 / 4 = 10.

Final Answer: The final answer to the original input question is 10. Great! Let's work on this problem together.

Question: How much is 8 + 2 / 2 * 2?

Thought: Okay, let's break this down step by step. First, we need to calculate the expression inside the parentheses. 2 / 2 = 1, so the equation becomes 8 + 2 * 1.

Action: Multiply 2 by 1.
Action Input: 2 * 1 = 2.

OBSERVATION: We have obtained 2 as the result of the multiplication.

Thought: Great, now we need to add 8 and 2.

Question: How much is 8 + 2?

Thought: Let's do the addition. 8 + 2 = 10.

Action: Add 8 and 2.
Action Input: 8 + 2 = 

[" Sure, I'd be happy to help you solve this equation! Here's the format you requested:\n\nQuestion: How much is 8 + 2 / 2 * 2?\n\nThought: Let's start by adding 8 and 2: 8 + 2 = 10.\n\nAction: Divide 10 by 2: 10 / 2 = 5.\n\nAction Input: 5.\n\nOBSERVATION: 5 is the result of dividing 10 by 2.\n\nThought: Great, now we have 5 as a result. Let's use it to simplify the equation: 8 + 2 / 2 * 2 = 8 + 2 / 4 = 10.\n\nFinal Answer: The final answer to the original input question is 10.",
 " Great! Let's work on this problem together.\n\nQuestion: How much is 8 + 2 / 2 * 2?\n\nThought: Okay, let's break this down step by step. First, we need to calculate the expression inside the parentheses. 2 / 2 = 1, so the equation becomes 8 + 2 * 1.\n\nAction: Multiply 2 by 1.\nAction Input: 2 * 1 = 2.\n\nOBSERVATION: We have obtained 2 as the result of the multiplication.\n\nThought: Great, now we need to add 8 and 2.\n\nQuestion: How much is 8 + 2?\n\nThought: Let's do the addition. 8 + 2 = 10.\n\nActio

In [ ]:
# SELF_REFLECT_PROMPT = """
# ### User input ###
# {initial_prompt}:
# ### Result ###
# {results}

# Reflect about the result regarding the user input and give the final answer
# """
# display_chat(
#     SELF_REFLECT_PROMPT.format(
#         initial_prompt=PROMPT_TEMPLATE.format(query="how much is 8 + 2 / 2 * 2?"),
#         results="\n".join(results)
#     )
# )
display_chat(PROMPT_TEMPLATE.format(query="how much is 8 + 2 / 2 * 2?"))